1. 데이터 로드

In [1]:
# 패키지 임포트
!pip install numpy==1.20.0
!pip install gensim==3.7.0 --user
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import re
from konlpy.tag import Okt
from io import BytesIO
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

You should consider upgrading via the 'c:\users\user\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\user\anaconda3\python.exe -m pip install --upgrade pip' command.


In [4]:
# 데이터를 데이터프레임으로 로드하고, 전체 문서의 수 출력

In [12]:
df1 = pd.read_csv('authordata_haveVector.csv')
df2 = pd.read_csv('bookdata_haveVector.csv')

In [13]:
# 상위 5행만 출력하여 확인

In [14]:
# 형태소 분석 하기 전 사용자 사전 추가
from ckonlpy.tag import Twitter

twitter = Twitter()
twitter.add_dictionary(list(df1['author']), 'Noun')
twitter.add_dictionary(list(df2['author']), 'Noun')
twitter.add_dictionary(list(df2['title']), 'Noun')

C:\Users\user\AppData\Roaming\Python\Python38\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [15]:
print(df1.head())
print(df2.head())

  author                                               info  \
0    나예리  특유의 냉소적인 캐릭터와 독특한 그림체로 데뷔 초기부터 나예리 매니아 집단을 형성한...   
1    이문열  년 서울에서 태어나 경북 영양 밀양 부산 등지에서 자랐다 서울대학교 사범대학에서 수...   
2  조애너 콜  저자 조애너 콜 은 어렸을 때부터 과학을 좋아하는 영리한 소녀였다 신기한 스쿨 버스...   
3    이호백  년 서울 출생 서울대학교 산업 미술학과와 고려대 신문방송학과 대학원을 졸업한 후 파...   
4    권정생  일본 도쿄에서 태어나 광복 직후 우리나라로 돌아왔다 경북 안동 일직면에서 마을 교회...   

                                            booklist  \
0  탐정이 될래요/네멋대로 해라!/아마도 달콤하겠지-MAY BE SO SWEET/특명!...   
1  사람의 아들/사람의 아들/황제를 위하여/금시조/선택/선택/젊은날의 초상/영웅시대/시...   
2                                    바퀴벌레/강아지가 태어났어요   
3                세상에서 제일 힘 센 수탉/도대체 그동안 무슨 일이 일어났을까?   
4  몽실 언니/사과나무 밭 달님/바닷가 아이들/점득이네/하느님의 눈물/밥데기 죽데기/또...   

                                             cleaned  \
0  특 유의 냉소 캐릭터 독특 그림 체 데뷔 초기 예리 매니아 집단 형성 작가 예리 파...   
1  서울 태어나다 경북 영양 밀양 부산 등지 자르다 서울대학교 사범대 학 수학 동아일보...   
2  저자 조 애너 콜 어리다 과학 좋다 영리 소녀 신기 스쿨 버스 프리 즐 선생님 처럼...   
3  서울 출생 서울대학교 산업 미술 학과 고려대 신문방송학 대학원 졸업 파리 대학 커뮤...   
4  일

In [16]:
def str_to_vector(str_vector):    
    str_vector=str_vector.replace('[','').replace(']','').split()
    list_vector = list(map(float, str_vector))
    vector=np.array(list_vector)    
    return vector

df2['vector']=df2['vector'].apply(str_to_vector)


0      [-0.000136356655, 0.000137033829, -0.000128305...
1      [-0.000117387448, 0.000114183233, -9.06481218e...
2      [0.000143364712, 4.75262095e-05, 4.52641652e-0...
3      [2.78678526e-05, 6.58732752e-05, 2.47672015e-0...
4      [3.84600899e-05, -7.59046816e-06, 4.11704932e-...
                             ...                        
134    [2.52884747e-05, 5.5673454e-06, -6.66477645e-0...
135    [-1.79595172e-05, 3.92592347e-06, 3.47515561e-...
136    [-2.40112495e-05, 1.05386716e-05, -4.15595532e...
137    [0.000129926892, -2.70366872e-05, 1.11418194e-...
138    [0.000138639385, 6.56343764e-05, 7.59905815e-0...
Name: vector, Length: 139, dtype: object


4. 추천 시스템 구현하기

In [19]:
# 각 벡터간 코사인 유사도

document_embedding_list=list(df2['vector'])
cosine_similarities = cosine_similarity(document_embedding_list, document_embedding_list)
print('코사인 유사도 매트릭스의 크기 :',cosine_similarities.shape)

코사인 유사도 매트릭스의 크기 : (139, 139)


In [30]:
print(df1.loc[df1['author'] == '이문열']['booklist'][1].split('/'))

['사람의 아들', '사람의 아들', '황제를 위하여', '금시조', '선택', '선택', '젊은날의 초상', '영웅시대', '시인', '오디세이아 서울', '선택', '호모 엑세쿠탄스', '이문열 중단편 전집', '사색', '시대와의 불화', '신들메를 고쳐매며', '변경', '대륙의 한', '삼국지', '수호지', '초한지']


In [20]:
# 코사인 유사도를 이용하여, 가장 설명이 유사한 5개의 작가을 찾아내는 함수 만들기

In [21]:
def recommendations_by_booklist(author):
    booklist=df1.loc[df1['author'] == author]['booklist'][1].split('/')
    
    booklist_vector=[]
    for book in booklist:d
        try:
            book_vector.append(df1.loc[df2['title'] == book]['vector'][1])
        except:
            print('일치하는 도서를 발견하지 못했습니다')

    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:6]

    book_indices = [i[0] for i in sim_scores]

    recommend = books.iloc[book_indices].reset_index(drop=True)

    for i in book_indices:
        print(books.loc[i]['author'])